<a href="https://colab.research.google.com/github/jingyuan0422/113-2-Programing-Language/blob/main/PL_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [188]:
# Authorize Google Drive
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [189]:
# Output from GoogleSheet “HW2_data”
import pandas as pd
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/16XTRDs5ZlWc5285Nq1kZHsrIaZh--NrfWvlTxdtA0Qs/edit?gid=0#gid=0')

from gspread_dataframe import set_with_dataframe
worksheets = gsheets.worksheets()

# Save all sheets
sheet_dfs = {}

for ws in worksheets:
    data = ws.get_all_values()
    df = pd.DataFrame(data[1:], columns=data[0])
    sheet_dfs[ws.title] = df

sheet1 = sheet_dfs['Sheet1']
sheet2 = sheet_dfs['Sheet2']
sheet3 = sheet_dfs['Sheet3']
sheet4 = sheet_dfs['Sheet4']

In [190]:
sheet1

,Entity,Code,Year,Share of contribution to global warming
0,Afghanistan,AFG,1851,0.15562129
1,Afghanistan,AFG,1852,0.15542437
2,Afghanistan,AFG,1853,0.15504095
3,Afghanistan,AFG,1854,0.15451188
4,Afghanistan,AFG,1855,0.15467837
...,...,...,...,...
40996,Zimbabwe,ZWE,2019,0.10586824
40997,Zimbabwe,ZWE,2020,0.104918584
40998,Zimbabwe,ZWE,2021,0.103950545
40999,Zimbabwe,ZWE,2022,0.103107095


In [191]:
"""Chart 1:
Contributions to the change in global mean surface temperature, 1851 to 2023
"""

# Select the target countries
target_entities = ['Brazil', 'China', 'European Union (27)', 'India', 'Russia', 'Taiwan', 'United Kingdom', 'United States']
df1_selected = sheet1[sheet1['Entity'].isin(target_entities)]

# Pivot analysis: Rows are years, columns are countries, and values are share
df1_pivot = df1_selected.pivot(index='Year', columns='Entity', values='Share of contribution to global warming')

df1_pivot = df1_pivot.apply(pd.to_numeric, errors='coerce')
df1_pivot = df1_pivot.round(2)

In [192]:
df1_pivot

Entity,Brazil,China,European Union (27),India,Russia,Taiwan,United Kingdom,United States
Year,,,,,,,,
1851,1.51,5.92,11.25,6.03,12.21,0.02,5.07,24.13
1852,1.35,5.63,11.24,5.83,11.96,0.02,4.99,24.58
1853,1.29,5.55,11.22,5.67,11.86,0.02,4.91,24.69
1854,1.27,5.30,11.22,5.55,11.97,0.02,5.11,24.86
1855,1.29,5.12,11.27,5.46,11.75,0.02,5.14,25.09
...,...,...,...,...,...,...,...,...
2019,5.05,12.02,10.67,4.62,6.89,0.31,2.54,17.71
2020,5.05,12.25,10.58,4.65,6.86,0.31,2.50,17.59
2021,5.04,12.48,10.49,4.69,6.84,0.31,2.47,17.47


In [205]:
import plotly as plt
import plotly.express as px
import plotly.graph_objects as go


# Creat chart 1
fig1 = go.Figure()

# Plot the line curve for each country
for entity in df1_pivot.columns:
    fig1.add_trace(go.Scatter(x=df1_pivot.index, y=df1_pivot[entity], mode='lines', name=entity))

# 設定標題和標籤
fig1.update_layout(
    title="Contributions to the change in global mean surface temperature",
    xaxis_title="Year",
    yaxis_title="Share of contribution to global warming (%)",
)

# 顯示圖表
fig1.show()

In [194]:
sheet2

,Entity,Code,Year,Annual CO₂ emissions
0,Afghanistan,AFG,1949,14656
1,Afghanistan,AFG,1950,84272
2,Afghanistan,AFG,1951,91600
3,Afghanistan,AFG,1952,91600
4,Afghanistan,AFG,1953,106256
...,...,...,...,...
29132,Zimbabwe,ZWE,2019,10262950
29133,Zimbabwe,ZWE,2020,8494503
29134,Zimbabwe,ZWE,2021,10203630
29135,Zimbabwe,ZWE,2022,10424940


In [195]:
sheet3

,Entity,Code,Year,Share of global annual CO₂ emissions
0,Afghanistan,AFG,1949,0.0002816276
1,Afghanistan,AFG,1950,0.0014210546
2,Afghanistan,AFG,1951,0.0014354152
3,Afghanistan,AFG,1952,0.0014163209
4,Afghanistan,AFG,1953,0.001597622
...,...,...,...,...
27323,Zimbabwe,ZWE,2019,0.02765975
27324,Zimbabwe,ZWE,2020,0.024182588
27325,Zimbabwe,ZWE,2021,0.027583541
27326,Zimbabwe,ZWE,2022,0.027953522


In [196]:
"""Chart 2 and 3:
Chart 2: 2023 CO₂ Emissions by Country
Chart 3: 2023 Share of Global CO₂ Emissions
"""

# Select the data for the year 2023
sheet2['Year'] = pd.to_numeric(sheet2['Year'], errors='coerce')
sheet3['Year'] = pd.to_numeric(sheet3['Year'], errors='coerce')

df2 = sheet2[sheet2['Year'] == 2023][['Entity', 'Annual CO₂ emissions']]
df3 = sheet3[sheet3['Year'] == 2023][['Entity', 'Share of global annual CO₂ emissions']]

df2['Annual CO₂ emissions'] = pd.to_numeric(df2['Annual CO₂ emissions'], errors='coerce')

# Merge using the Entity column
merged_df = pd.merge(df2, df3, on='Entity', how='inner')

# Select the target countries
df2_selected = merged_df[merged_df['Entity'].isin(target_entities)]

# Convert units
df2_selected.loc[:, 'Annual CO₂ emissions'] = df2_selected['Annual CO₂ emissions'] / 1e9  # Turn to billion
df2_selected.loc[:, 'Annual CO₂ emissions'] = df2_selected['Annual CO₂ emissions'].astype(float).round(1)
df2_selected.loc[:, 'Share of global annual CO₂ emissions'] = df2_selected['Share of global annual CO₂ emissions'].astype(float).round(1)

# Sorted
df2_selected = df2_selected.sort_values(by='Annual CO₂ emissions', ascending=False)

In [197]:
df2_selected

,Entity,Annual CO₂ emissions,Share of global annual CO₂ emissions
44,China,11.9,31.5
223,United States,4.9,13.0
99,India,3.1,8.1
74,European Union (27),2.5,6.6
176,Russia,1.8,4.8
31,Brazil,0.5,1.3
207,Taiwan,0.3,0.7
222,United Kingdom,0.3,0.8


In [198]:
# Draw a bar chart
fig2 = px.bar(df2_selected, x='Entity', y='Annual CO₂ emissions',
              title='2023 CO₂ Emissions by Country',
              labels={'2023 CO₂ Emissions by Country': 'Emissions (billion)', 'Entity': 'Entity'})

fig2.update_traces(text=df2_selected['Annual CO₂ emissions'], textposition='outside')

# Show the figure
fig2.show()

In [199]:
# Calculate "Others"
others_share = 100 - pd.to_numeric(df2_selected['Share of global annual CO₂ emissions'], errors='coerce').sum()
others_share.round(1)

# Add a column "Others"
df_pie = df2_selected[['Entity', 'Share of global annual CO₂ emissions']].copy()
df_pie.loc[len(df_pie)] = ['Others', others_share]

# Draw a pie chart
fig3 = px.pie(df_pie, names='Entity', values='Share of global annual CO₂ emissions',
              title='2023 Share of Global CO₂ Emissions',
              labels={'Entity': 'Entity', 'Share of global annual CO₂ emissions': 'Share (%)'})

fig3.show()

In [200]:
sheet4

,Entity,Code,Year,"GDP per capita, PPP (constant 2021 international $)"
0,Afghanistan,AFG,2000,1617.8264
1,Afghanistan,AFG,2001,1454.1108
2,Afghanistan,AFG,2002,1774.3087
3,Afghanistan,AFG,2003,1815.9282
4,Afghanistan,AFG,2004,1776.9182
...,...,...,...,...
7058,Zimbabwe,ZWE,2019,3294.8062
7059,Zimbabwe,ZWE,2020,2987.2698
7060,Zimbabwe,ZWE,2021,3184.7854
7061,Zimbabwe,ZWE,2022,3323.1218


In [201]:
"""Chart 4
GDP vs CO₂ Emissions by Country
"""


# Select the data for the year 2023
sheet4['Year'] = pd.to_numeric(sheet4['Year'], errors='coerce')
df4 = sheet4[sheet4['Year'] == 2023][['Entity', 'GDP per capita, PPP (constant 2021 international $)']]

# Merge using the Entity column
merged_df = pd.merge(df2, df4, on='Entity', how='inner')

# Select the target countries
df4_selected = merged_df[merged_df['Entity'].isin(target_entities)]

# Convert units
df4_selected.loc[:, 'Annual CO₂ emissions'] = df4_selected['Annual CO₂ emissions'] / 1e9  # Turn to billion
df4_selected.loc[:, 'Annual CO₂ emissions'] = df4_selected['Annual CO₂ emissions'].astype(float).round(1)
df4_selected.loc[:, 'GDP per capita, PPP (constant 2021 international $)'] = \
  df4_selected['GDP per capita, PPP (constant 2021 international $)'].astype(float).round(1)

# Sorted
df4_selected = df4_selected.sort_values(by='Annual CO₂ emissions', ascending=False)

In [202]:
df4_selected

,Entity,Annual CO₂ emissions,"GDP per capita, PPP (constant 2021 international $)"
36,China,11.9,22137.6
186,United States,4.9,74577.5
80,India,3.1,9160.1
59,European Union (27),2.5,54446.9
146,Russia,1.8,39753.5
24,Brazil,0.5,19018.2
185,United Kingdom,0.3,54542.0


In [204]:
# Draw a scatter plot
fig3 = px.scatter(df4_selected,
                 x='GDP per capita, PPP (constant 2021 international $)',
                 y='Annual CO₂ emissions',
                 text='Entity',
                 title='GDP vs CO₂ Emissions by Country',
                  labels={'GDP per capita, PPP (constant 2021 international $)': 'GDP per Capita (USD)',
                          'Annual CO₂ emissions': 'Annual CO₂ Emissions (billion tons)'})

fig3.update_traces(textposition='bottom center')

fig3.show()